# Introduction
We will be employing the linear regression model in order to predict house sale prices in Ames, Iowa based on various attributes of the houses. This analysis will require cleaning, transforming, and selecting features in our dataset. Explanations of the features are listed on https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt

# Preparation and Data Exploration

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

housing = pd.read_csv('AmesHousing.tsv', sep = '\t')

housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


It will be useful knowing what types the values in each column are.

In [143]:
housing.dtypes

Order               int64
PID                 int64
MS SubClass         int64
MS Zoning          object
Lot Frontage      float64
Lot Area            int64
Street             object
Alley              object
Lot Shape          object
Land Contour       object
Utilities          object
Lot Config         object
Land Slope         object
Neighborhood       object
Condition 1        object
Condition 2        object
Bldg Type          object
House Style        object
Overall Qual        int64
Overall Cond        int64
Year Built          int64
Year Remod/Add      int64
Roof Style         object
Roof Matl          object
Exterior 1st       object
Exterior 2nd       object
Mas Vnr Type       object
Mas Vnr Area      float64
Exter Qual         object
Exter Cond         object
                   ...   
Bedroom AbvGr       int64
Kitchen AbvGr       int64
Kitchen Qual       object
TotRms AbvGrd       int64
Functional         object
Fireplaces          int64
Fireplace Qu       object
Garage Type 

# Feature Engineering

We will be writing and using the functions transform_features(), select_features(), and train_and_test(), as the use of these functions in our workflow will be how we obtain optimal fit parameters to the linear regression function for our dataset.

We will remove features with many missing values, remove columns that leak information about the sale, transform features into proper numerical formats, and create new features by combining other features. This will allow the numerical values in the column to have a relationship with the housing sale prices and be used in the training of the linear regression model.

In [226]:
import math

def transform_features(train):
    # Drop Order and PID - These dentification labels, unique to each item, are not useful for machine learning
    train = train.drop(columns = ['Order', 'PID'])
    
    # Obtain list of features - Drop 'SalePrice', the target
    features = train.columns.drop('SalePrice')
    
    # Determine the cutoff for missing values, drop columns with amount above cutoff
    null_cutoff = math.ceil(train.shape[0] * .05)
    null_features = features[(train[features].isnull().sum() >= null_cutoff)]
    new_train = train.drop(columns = null_features)
    
    # Create new features by combining other features
    new_train['Years Since Remod'] = new_train['Yr Sold'] - train['Year Remod/Add']
    new_train['Years Before Sale'] = new_train['Yr Sold'] - train['Year Built']
    
    # Remove rows where these new columns have negative values - No logical sense
    neg_remod = new_train[new_train['Years Since Remod'] < 0].index
    new_train = new_train.drop(neg_remod, axis = 0)
    neg_sell = new_train[new_train['Years Before Sale'] < 0].index
    new_train = new_train.drop(neg_sell, axis = 0)
    
    # Drop columns that leak information about the sale
    # The aim of this model is to predict sales prices of houses that are hypothetically not sold
    new_train = new_train.drop(columns = ['Mo Sold', 'Yr Sold', 'Sale Type', 'Sale Condition', 'Year Built',
                                          'Year Remod/Add'])
    
    # Address columns that still have missing values
    # Specifically, less than 5% of the column values are missing. 
    # For numerical, replace with most common value (mode)
    features = new_train.columns.drop('SalePrice')      # Columns got dropped previously
    null_thresh = math.ceil(new_train.shape[0] * 0.05)
    null_features = features[(new_train[features].isnull().sum() < null_thresh) & 
                             (new_train[features].isnull().sum() > 0)]
    
    for col in null_features:
        if new_train[col].dtypes == 'float64' or new_train[col].dtypes == 'int64':
            new_train[col] = new_train[col].fillna(new_train[col].mode()[0])
    
    return new_train

def select_features(train, corr_cutoff = 0.4, unique_max = 10):    
    # First, filter out the numerical features with low correlation with SalePrice
    corrmat = (train.select_dtypes(include = ['int64', 'float64'])).corr()
    sorted_corrs = np.abs(corrmat['SalePrice'].sort_values())
    # Keep only numerical features with strong correlation with SalePrice. Cutoff is determined through experiment.
    strong_coors = sorted_corrs[sorted_corrs > corr_cutoff]
    
    # Keep only the features with strong correlation with SalePrice
    new_train = train[strong_coors.index]
    # Bring back the string columns
    new_train = pd.concat((new_train, train.select_dtypes(include = 'object')), axis = 1)
    
    # List nominal features, listed on https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", 
                        "Neighborhood", "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", 
                        "Roof Matl", "Exterior 1st", "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", 
                        "Central Air", "Garage Type", "Misc Feature", "Sale Type", "Sale Condition"]
    
    # Some of the nominal features were dropped in the previous function
    for col in nominal_features:
        if col in new_train.columns:
            category_values = new_train[col].value_counts()
            
            # Drop the column if there are more than unique_max unique values
            # or if more than 95% of the values in a column belong to a specific category
            if len(category_values) > unique_max or (category_values[0]/new_train.shape[0]) > 0.95:
                new_train = new_train.drop(columns = col)
            else:
                # Otherwise, convert the nominal feature into dummy columns
                dummy_cols = pd.DataFrame()
                new_train[col] = new_train[col].astype('category')
                dummy_cols = pd.get_dummies(new_train[col])
                new_train = pd.concat((new_train, dummy_cols), axis = 1)
                new_train = new_train.drop(columns = col)
    
    # Convert remaining string columns into categorical columns
    dummy_cols = pd.DataFrame()
    text_cols = new_train.select_dtypes(include = ['object']).columns
    
    for col in text_cols:
        new_train[col] = new_train[col].astype('category')
        dummy_cols = pd.get_dummies(new_train[col])
        new_train = pd.concat((new_train, dummy_cols), axis = 1)
        new_train = new_train.drop(columns = col)
    
    return new_train

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score

def train_and_test(data, k = 0):
    # Get a dataframe from select_features, returning our features and target
    features = data.select_dtypes(include = ['integer', 'float'])
    # Exclude target column
    features = features.drop(columns = 'SalePrice')
    # Get list of feature columns
    feature_cols = features.columns
    
    # Instantiate the linear regression model
    lr = LinearRegression()
    
    if k == 0:
        # Perform holdout implementation
        train = data.iloc[:1460]
        test = data.iloc[1460:]
        
        # Train the linear regression model, use on test
        lr.fit(train[feature_cols], train['SalePrice'])
        prediction = lr.predict(test[feature_cols])

        # Calculate root mean square error
        rmse = mean_squared_error(test['SalePrice'], prediction) ** .5
        
    elif k == 1:
        data = data.reindex(np.random.permutation(data.index))
        
        fold_one = data.iloc[:1460]
        fold_two = data.iloc[1460:]
        
        # Train LR model on fold_one, test on fold_two. Calculate rmse
        lr.fit(fold_one[feature_cols], fold_one['SalePrice'])
        prediction = lr.predict(fold_two[feature_cols])
        rmse_1 = mean_squared_error(fold_two['SalePrice'], prediction) ** .5
        
        # Repeat by training on fold_two, testing on fold_one
        lr.fit(fold_two[feature_cols], fold_two['SalePrice'])
        prediction = lr.predict(fold_one[feature_cols])
        rmse_2 = mean_squared_error(fold_one['SalePrice'], prediction) ** .5
        
        rmse = .5 * (rmse_1 + rmse_2)
        
    else:
        # Implement k-fold cross-validation using k folds
        
        kf = KFold(n_splits = k, shuffle = True, random_state = 1)
        
        # Calculate the average root mean square error by training and testing the LR model for each fold
        mses = cross_val_score(lr, data[feature_cols], data['SalePrice'], 
                               scoring = 'neg_mean_squared_error', cv = kf)
        rmses = np.sqrt(np.abs(mses))
        rmse = np.mean(rmses)
    
    return rmse

transform_df = transform_features(housing)
filtered_df = select_features(transform_df, .3)

rmses = {}

for key in [0, 1, 2, 4, 5, 10, 15, 25]:
    rmses[key] = train_and_test(filtered_df, key)
    
pd.Series(rmses)

0     32223.736062
1     29951.510226
2     29037.991265
4     27992.630718
5     27915.561741
10    27509.067860
15    27107.294381
25    26472.300605
dtype: float64